In [13]:
import os
import pycardano as pc
import random
import time
from vesting import *
import pathlib

In [15]:
# Copy your BlockFrost project ID below. Go to https://blockfrost.io/ for more information.
# BLOCK_FROST_PROJECT_ID = "your_project_id"
NETWORK = pc.Network.TESTNET

# Use ogmios backend for chain context
context = pc.OgmiosChainContext(ws_url="wss://ogmios-preprod-api-testing-2fdd06.us1.demeter.run", network=NETWORK)

# chain_context = BlockFrostChainContext(
#     project_id=BLOCK_FROST_PROJECT_ID,
#     base_url=ApiUrls.preprod.value,
# )

"""Preparation"""
# Define the root directory where images and keys will be stored.
PROJECT_ROOT = "marketplace"
root = pathlib.Path(PROJECT_ROOT)

# Create the directory if it doesn't exist
root.mkdir(parents=True, exist_ok=True)

"""Generate keys"""
key_dir = root / "keys"
key_dir.mkdir(exist_ok=True)

# Load payment keys or create them if they don't exist
def load_or_create_key_pair(base_dir, base_name):
    skey_path = base_dir / f"{base_name}.skey"
    vkey_path = base_dir / f"{base_name}.vkey"
    stake_skey_path = base_dir / f"{base_name}_stake.skey"
    stake_vkey_path = base_dir / f"{base_name}_stake.vkey"

    if skey_path.exists():
        skey = pc.PaymentSigningKey.load(str(skey_path))
        vkey = pc.PaymentVerificationKey.from_signing_key(skey)
        stake_skey = pc.StakeSigningKey.load(str(stake_skey_path))
        stake_vkey = pc.StakeVerificationKey.from_signing_key(stake_skey)
    else:
        key_pair = pc.PaymentKeyPair.generate()
        key_pair.signing_key.save(str(skey_path))
        key_pair.verification_key.save(str(vkey_path))
        stake_key_pair = pc.StakeKeyPair.generate()
        stake_key_pair.signing_key.save(str(stake_skey_path))
        stake_key_pair.verification_key.save(str(stake_vkey_path))
        stake_skey = stake_key_pair.signing_key
        stake_vkey = stake_key_pair.verification_key
        skey = key_pair.signing_key
        vkey = key_pair.verification_key
    
    return skey, vkey, stake_skey, stake_vkey

In [16]:
owner_skey, owner_vkey, owner_stake_skey, owner_stake_vkey = load_or_create_key_pair(key_dir, "owner")
beneficiary_skey, beneficiary_vkey, beneficiary_stake_skey, beneficiary_stake_vkey = load_or_create_key_pair(key_dir, "bidder")
owner_address = pc.Address(payment_part=owner_vkey.hash(), staking_part=owner_stake_vkey.hash(), network=NETWORK)
beneficiary_address = pc.Address(payment_part=beneficiary_vkey.hash(), staking_part=beneficiary_stake_vkey.hash(), network=NETWORK)
print(f"Owner address: {owner_address}")
print(f"beneficiary address: {beneficiary_address}")

Owner address: addr_test1qq5g0m0h7kzts8mfuckeuyr2qxk9fgfskz47kx0fz7l45m35sz43s8xuc9qt8kdxlchhy90d4yp0sa43vdrxuu9jewps34ue95
beneficiary address: addr_test1qzjr9whvuyldxf9ft6998jjwnq72syr2yeqlmygxfhl2d2c8lxhg9ar5qfnk2w5j0gj6fvygkdcdgd9e7gftjgt4dvhqv0ej3s


In [17]:
script_path = pathlib.Path().cwd()
script_path = script_path.joinpath("build","vesting", "script.cbor")

In [18]:
with open(script_path) as f:
        cbor_hex = f.read()

cbor = bytes.fromhex(cbor_hex)

In [8]:
plutus_script = pc.PlutusV2Script(cbor)
script_hash = pc.plutus_script_hash(plutus_script)
script_address = pc.Address(script_hash, network=NETWORK)

In [19]:
# Get payment address    
payment_address = beneficiary_address 

In [20]:
# Find a script UTxO
utxo_to_spend = None
for utxo in context.utxos(str(script_address)):
    if utxo.output.datum:
            
        try:
                params = VestingParams.from_cbor(utxo.output.datum.cbor)
                print(params)
                
        except Exception:
                continue
        
        if (
                params.beneficiary == bytes(payment_address.payment_part) and
                params.deadline < time.time() * 1000
        
        ):
                utxo_to_spend = utxo
                break
assert isinstance(utxo_to_spend, pc.UTxO), "No script UTxOs found!"

VestingParams(beneficiary=b'\xa42\xba\xec\xe1>\xd3$\xa9^\x8aS\xcaN\x98<\xa8\x10j&A\xfd\x91\x06M\xfe\xa6\xab', deadline=1680226821000)


In [22]:
# Find a collateral UTxO
non_nft_utxo = None
for utxo in context.utxos(str(payment_address)):
    # multi_asset should be empty for collateral utxo
    if not utxo.output.amount.multi_asset and utxo.output.amount.coin > 5000000:
        non_nft_utxo = utxo
        break
assert isinstance(non_nft_utxo, pc.UTxO), "No collateral UTxOs found!"

In [39]:
# Need a redeemer
redeemer = pc.Redeemer(pc.RedeemerTag.SPEND, 0)

In [40]:
# Build the transaction
builder = pc.TransactionBuilder(context)
builder.add_script_input(utxo_to_spend, script=plutus_script, redeemer=redeemer)
builder.collaterals.append(non_nft_utxo)
# This tells pycardano to add vkey_hash to the witness set when calculating the transaction cost
vkey_hash: pc.VerificationKeyHash = payment_address.payment_part
builder.required_signers = [vkey_hash]
# we must specify at least the start of the tx valid range in slots
builder.validity_start = context.last_block_slot
# This specifies the end of tx valid range in slots
builder.ttl = builder.validity_start + 1000

In [41]:
# Sign the transaction
signed_tx = builder.build_and_sign(
    signing_keys=[beneficiary_skey],
    change_address=payment_address,
)

In [42]:
# Submit the transaction
context.submit_tx(signed_tx.to_cbor())

# context.submit_tx(signed_tx.to_cbor())
print(f"transaction id: {signed_tx.id}")
print(f"Cardanoscan: https://preprod.cardanoscan.io/transaction/{signed_tx.id}")

transaction id: d84be2c935384691bfbcca80a660ceaa5d3e332e59994a5edf162a112c612a69
Cardanoscan: https://preprod.cardanoscan.io/transaction/d84be2c935384691bfbcca80a660ceaa5d3e332e59994a5edf162a112c612a69
